# Recurrent Neural Networks Without ML Library

In 2018 Google I/O, Google introduced a feature which can be used to book your appointments through a single call. You just have to tell the google assistant that when and where you want to book a reservation/apointment then assistant will call the respected authority, interact with them and will tell you conclusion of the discussion. This feature of assistant is what google calls as <b>Google Duplex</b>. For this feature, assistant has to remember whole context during the call and for that google has used the Recurrent Neural Networks. RNNs are great if your present output depends upon present and past inputs/outputs as well. Now let us discuss basics about RNN,

A simple and basic RNN is drawn in following figure,

<img src="./Images/RNNReprentation.png">

Do you find any difference between A simple Feedforward network and Recurrent network by observing just above diagram ? Yes! Instead of just forward propagating through weights and input we will consider the previous hidden state as well while calculating the output of the layer. So the arrows of the hidden layers are pointing to the output and itself also.

But why we should consider the previous hidden state ?

Well consider the trained feedforward network, the hidden state of that network represents all the inputs on which it has learned while training and gives the output accordingly. In RNN your current state output does depends upon your current input and past inputs. So if we provide the previous hidden state while calculating the current output then its like considering all previous inputs.

Let us consider the 3 layered Rnn (I/p, Hidden, Output) as shown in below figure and discuss cases for forward propagation and backward propagation,
Now for forward propagation we have following equations,

<img src="./Images/3_layer_RNN.png" style="height:500px">

output_of_layer1 = $g$( inputs  * weights $_{(W1)}$ + previous hidden state$ * W_{hh})$

Output_of_layer2 = $g($output_of_layer1 * W2)

Where,
g(x) is a activation function <br/>
W1 - Weights Input-hidden <br/>
$W_{hh}$ - Hidden to hidden matrix

Here Output_of_layer2 is the output of our network. While training we will compare that output with the actual output, calculate error/loss, backpropagate through our network to update weights.

Now lets write the equation for updating the weights while backpropagating,

W2 = W2 + Gradient of output of Network * Output_of_layer1

Gradient of output of Network is calculated by multiplying output error with the derivation of the activation function at Layer2 output value.<br/>
Now for updating the weights of I/p Layer to hidden layer we have to calculate the gradient of the output produced by the layer_1 lets call it as layer_1_gradients. and the equation for calculting the gradient is,

layer_1_gradients = future_layer1_gradient * W$_{hh}$ + Gradient of output of Network * W2 * Gradient of output of layer_1

The future layer1 gradients are the values of the layer_1_gradients 1 timestep back while backpropagation.<br/>
Then we update the weights W1 as,

W1 = W1 + inputs * layer_1_gradients

Now we just have to update the Whh matrix as follows,

$W_{hh} = W_{hh}$ + previous_hidden_layer * layer_1_gradients

Here previous_hidden_layer is the previous time step hidden state.

We will consider the addition of two numbers bit by bit. During addition if both the bits are 1 then addition is 0 and carry is 1, that carry is carried forward for next input to add. So current input is dependent on output of the previous addition. We can use RNN for the problem. We will use the same network as shown is above diagram except in the hidden layer we will use 16 neurons. So that our input layer have 2 input neurons, hidden layer have 16 neurons and output layer has only 1 neuron.

We will be needing labeled data for the training, for that we will add two numbers and consider the addition as our label/Expected output.
Steps for training,<br/>
1. Consider two 8 bit binary numbers.
2. Take 1 bit from each number from RHS and give input to the network.
3. Calculate output through forward propagation
4. While calculating the output save the hidden state as it will be used for Further calculations
5. Repeat Step 1,2,3,4 for 8 times as we are considering addition for 2 8 bit numbers.
6. After 5 Now we have to do backpropagation as 2 8 bit numbers are added by the network.
7. For backpropagation calculate necessary values and also save the gradients as it will be used for further calculations.
8. Repeat step 7 8 times.
9. The steps 1-8 completes 1 iteration, so repeat those steps 10000 times and our network will train.

Now coding time,

In [1]:
import copy
import numpy as np

In [2]:
binary_values = {}
no_of_bits_binary = 8
max_val = (2**no_of_bits_binary)
for i in range(max_val):
    binary_values[i] = np.unpackbits(np.array([i],dtype=np.uint8).T)

In [3]:
# sigmoid function forward propagation
def forwardPropagate(x):
    return 1/(1+np.exp(-x))
# sigmoid derivative backward propagation
def backPropagate(x):
    return x*(1-x)

In [4]:
#hyperparameters
inputLayerSize = 2
hiddenLayerSize = 16
outputLayerSize = 1

W1 = 2 * np.random.random((inputLayerSize,hiddenLayerSize)) - 1
W2 =  2 * np.random.random((hiddenLayerSize,outputLayerSize)) - 1
Whh =  2 * np.random.random((hiddenLayerSize,hiddenLayerSize)) - 1

#For updating these values we will require temporary variables with same size
W1_update = np.zeros_like(W1)
W2_update = np.zeros_like(W2)
Whh_update = np.zeros_like(Whh)

#We need labeled data for labels we will simply add two numbers and store its output as the label
#10000 is the number of iterations
for i in range(10000):
    a_int = np.random.randint(max_val/2)
    b_int = np.random.randint(max_val/2)
    a = binary_values[a_int]
    b = binary_values[b_int]
    c = binary_values[a_int + b_int]
    
    d = np.zeros_like(c)
    overall_error = 0
    
    output_layer_gradients = list()
    hidden_layer_gradients = list()
    hidden_layer_values = list()
    hidden_layer_values.append(np.zeros(hiddenLayerSize))
    
    #Forward Propagation throug our rnn for each binary digit
    for position in range(no_of_bits_binary):
        #Input X will take 1 bit from a and 1 bit from b
        X = np.array([[a[no_of_bits_binary - position - 1],b[no_of_bits_binary - position - 1]]])
        #Output label 
        y = np.array([[c[no_of_bits_binary - position - 1]]]).T
        
        #Forward Propagation through layer 1
        layer_1 = forwardPropagate(np.dot(X,W1)+np.dot(hidden_layer_values[-1],Whh))
        #layer_2 is the predicted output of the network
        layer_2 = forwardPropagate(np.dot(layer_1,W2))
        #calculate error by subtracting actual value - predicted value
        output_error = y - layer_2
        
        #calculate gradient after each forward propagation
        output_layer_gradients.append((output_error)*backPropagate(layer_2))
        
        overall_error += np.abs(output_error[0])
        
        #rondoff the values
        d[no_of_bits_binary - position - 1] = np.round(layer_2[0][0])
        
        #Saving the hidden State values
        hidden_layer_values.append(copy.deepcopy(layer_1))
    future_layer1_gradient = np.zeros(hiddenLayerSize)
    
    #Now backpropagation
    for position in range(no_of_bits_binary):
        X = np.array(a[position],b[position])
        layer_1 = hidden_layer_values[-position-1]
        prev_hidden_layer = hidden_layer_values[-position-2]
        output_layer_gradient = output_layer_gradients[-position-1]
        layer_1_gradients = (future_layer1_gradient.dot(Whh.T) + output_layer_gradient.dot(W2.T)) * backPropagate(layer_1)
        
        #Updating all the values
        W2_update += np.atleast_2d(layer_1).T.dot(output_layer_gradient)
        Whh_update += np.atleast_2d(prev_hidden_layer).T.dot(layer_1_gradients)
        W1_update += X.T.dot(layer_1_gradients)
        
        future_layer1_gradient = layer_1_gradients
    #Update all the weights
    W1 += W1_update
    W2 += W2_update
    Whh += Whh_update
    
    #Clear the updated values
    W1_update *= 0
    W2_update *= 0
    Whh_update *= 0
    if (i % 1000 == 0):
        out=0
        print("Error:" + str(overall_error))
        print("Predicted "+str(d))
        print("Expected: "+str(c))
        for index,bit in enumerate(reversed(d)):
             out += bit * pow(2, index)
        print(str(a_int)+ " + "+str(b_int) + " = "+str(out))

Error:[3.53572324]
Predicted [0 0 0 0 0 0 0 0]
Expected: [0 1 1 0 0 1 0 0]
1 + 99 = 0
Error:[1.22344201]
Predicted [0 0 1 0 0 0 1 1]
Expected: [0 0 1 0 0 0 1 1]
16 + 19 = 35
Error:[3.25544081]
Predicted [0 1 1 1 1 1 1 0]
Expected: [1 1 1 0 1 0 1 0]
110 + 124 = 126
Error:[1.82797663]
Predicted [0 1 1 1 0 0 1 1]
Expected: [0 1 1 1 0 1 1 1]
11 + 108 = 115
Error:[0.85373709]
Predicted [1 0 1 0 1 1 1 1]
Expected: [1 0 1 0 1 1 1 1]
91 + 84 = 175
Error:[1.36818582]
Predicted [0 1 0 1 1 0 1 0]
Expected: [0 1 0 1 1 0 1 0]
61 + 29 = 90
Error:[1.93960733]
Predicted [0 1 0 1 1 1 1 0]
Expected: [0 1 0 1 1 0 0 0]
73 + 15 = 94
Error:[0.22906529]
Predicted [1 1 0 1 0 0 0 1]
Expected: [1 1 0 1 0 0 0 1]
112 + 97 = 209
Error:[0.28318212]
Predicted [0 0 1 1 0 1 1 1]
Expected: [0 0 1 1 0 1 1 1]
11 + 44 = 55
Error:[0.16649797]
Predicted [0 1 0 1 0 1 0 1]
Expected: [0 1 0 1 0 1 0 1]
51 + 34 = 85


As you can see the error gets minimized as training iteration proceeds further. If we give two numbers to our network then we can get result as follows,

In [14]:
def AddUsingRNN(first_number , second_number):
    a = binary_values[first_number]
    b = binary_values[second_number]
    additionBinary = np.zeros_like(b)
    out = 0
    hidden_layer = list() 
    hidden_layer.append(np.zeros(hiddenLayerSize))
    for position in range(no_of_bits_binary):
        X = np.array([a[no_of_bits_binary - position - 1],b[no_of_bits_binary - position - 1]])
        first_layer = forwardPropagate(np.dot(X,W1)+np.dot(hidden_layer[-1],Whh))
        second_layer = forwardPropagate(np.dot(first_layer,W2))
        hidden_layer.append(first_layer)
        additionBinary[no_of_bits_binary - position - 1] = np.round(second_layer)
    for index, bit in enumerate(reversed(additionBinary)):
        out += bit * pow(2, index)
    print(str(first_number) + " + " + str(second_number) + " = " + str(out))
    print("First Number "+str(a))
    print("Second Number"+str(b))
    print("Addition     "+str(additionBinary))

In [17]:
AddUsingRNN(4,28)

4 + 28 = 32
First Number [0 0 0 0 0 1 0 0]
Second Number[0 0 0 1 1 1 0 0]
Addition     [0 0 1 0 0 0 0 0]


For forward propagation and getting the output from the network we have to pass two numbers bit by bit to our network, after each pass we have to store the hidden state of the network as it will be used to findout the output at next timestep. If you see the output then at first carry is generated by adding 1 and 1 at 3rd position from RHS and it is forwarded till 7th position. We can say that our network has learned to add two numbers with carry forward.

The main use of RNN is in language models as in language models understanding the context is important for better results. I hope that you understood the basics behind the RNN and why google has used RNN in duplex feature of google assistant.